In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
import random
import math
import string
import re
from collections import Counter
import pickle

In [3]:
import konlpy
from konlpy.tag import Komoran

# DTM (Document-Term Matrix)

In [4]:
df = pd.read_pickle("2021_bow_df_exclude_frequent")
df

,회사명,사업 내용 bow
0,3S,"[(웨이퍼, 52), (반도체, 31), (시험, 22), (칼로리, 16), (메..."
1,AJ네트웍스,"[(렌탈, 129), (파렛트, 35), (아이템, 21), (고소, 17), (물..."
2,AK홀딩스,"[(화장품, 42), (항공, 42), (석유화학, 35), (애경, 29), (용..."
3,APS홀딩스,"[(지주회사, 13), (부동산, 7), (소프트웨어, 6), (정밀, 6), (주..."
4,AP시스템,"[(반도체, 48), (패널, 20), (양산, 9), (디스플레이, 7), (삼성..."
...,...,...
2828,흥국화재,"[(보험, 78), (보험료, 35), (손해보험, 32), (운용, 17), (신..."
2829,흥아해운,"[(해운, 36), (선박, 17), (운송, 9), (선사, 7), (과세, 7)..."
2830,희림,"[(공사, 12), (감리, 7), (건축, 6), (도급액, 6), (서비스업, ..."
2831,힐세리온,"[(초음파, 167), (진단, 130), (영상, 48), (혈류, 48), (패..."


In [5]:
# bow를 list로 불러오기
bow = []
with open("2021_bow.txt", "r") as f:
    for line in f:
        bow.append(str(line.strip()))
print(len(bow))
bow

34038


['가감',
 '가게',
 '가격결정',
 '가격상한',
 '가격연동제',
 '가격차별',
 '가격파괴',
 '가격표',
 '가결',
 '가경',
 '가경제',
 '가계',
 '가계도',
 '가계부',
 '가계부채',
 '가고',
 '가고시마',
 '가공량',
 '가공력',
 '가공료',
 '가공식품',
 '가공업자',
 '가공품',
 '가교',
 '가구',
 '가구원',
 '가군',
 '가규',
 '가금',
 '가까이',
 '가나',
 '가남읍',
 '가내',
 '가내수공업',
 '가노',
 '가늠',
 '가늠자',
 '가능',
 '가능동',
 '가니',
 '가니쉬',
 '가다랑어',
 '가닥',
 '가담',
 '가대',
 '가덕도',
 '가도',
 '가독',
 '가독성',
 '가동',
 '가두',
 '가드',
 '가드너',
 '가드레일',
 '가득',
 '가든',
 '가든시티',
 '가든파이브',
 '가디언',
 '가디언즈',
 '가라오케',
 '가락',
 '가락동',
 '가락시장',
 '가람',
 '가랑비',
 '가랑이',
 '가래',
 '가려움',
 '가려움증',
 '가로',
 '가로등',
 '가로변',
 '가로세로',
 '가로수',
 '가로수길',
 '가로수로',
 '가루',
 '가르가넬리',
 '가름',
 '가리',
 '가리개',
 '가리비',
 '가리온',
 '가마',
 '가마솥',
 '가망',
 '가맹',
 '가맹점',
 '가면',
 '가명',
 '가모',
 '가몬',
 '가문',
 '가문의영광',
 '가물',
 '가뭄',
 '가미',
 '가바',
 '가발',
 '가방',
 '가변',
 '가변밸브타이밍',
 '가변수',
 '가변저항',
 '가보',
 '가보세',
 '가봉',
 '가부',
 '가비',
 '가비아',
 '가사',
 '가사지원로봇',
 '가산',
 '가산금',
 '가산동',
 '가산면',
 '가상',
 '가상공간',
 '가상머신',
 '가상사설망',
 '가상세계',
 '가상키보드',
 '가상학습',
 '가상현실',
 '가상화',
 '가

In [31]:
dtm = pd.DataFrame(0, index=np.arange(len(df)), columns = ['Firm'] + bow)
dtm['Firm'] = df['회사명']
dtm

,Firm,가감,가게,가격결정,가격상한,가격연동제,가격차별,가격파괴,가격표,가결,...,힐,힐러리,힐링,힐스,힐스테이트,힐튼,힘,힘겨루기,힘줄,힙합
0,3S,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AJ네트웍스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AK홀딩스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,APS홀딩스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AP시스템,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2828,흥국화재,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2829,흥아해운,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2830,희림,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2831,힐세리온,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
for i in range(len(dtm)):
    for j in df.loc[i, '사업 내용 bow']:
        dtm.loc[i, j[0]] = j[1]

In [39]:
# column 개수가 너무 많아서 excel 파일로 저장 불가. 향후 재부팅 후 작업을 위해 pickle만 가능.
dtm.to_pickle("2021_dtm")

In [6]:
dtm = pd.read_pickle("2021_dtm")
dtm

,Firm,가감,가게,가격결정,가격상한,가격연동제,가격차별,가격파괴,가격표,가결,...,힐,힐러리,힐링,힐스,힐스테이트,힐튼,힘,힘겨루기,힘줄,힙합
0,3S,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AJ네트웍스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AK홀딩스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,APS홀딩스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AP시스템,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2828,흥국화재,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2829,흥아해운,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2830,희림,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2831,힐세리온,0,0,0,0,0,0,0,0,0,...,14,0,0,0,0,0,1,0,0,0


In [41]:
# JPE 페이퍼처럼 0 or 1
dtm2 = dtm.copy()
for i in range(len(dtm2)):
    for j in df.loc[i, '사업 내용 bow']:
        dtm2.loc[i, j[0]] = 1

In [ ]:
# bow 단어가 전혀 들어가 있지 않은 기업은 제외

In [ ]:
empty = []
for i in range(len(dtm2)):
    if sum(dtm2.loc[i].values[1:]) == 0:
        empty.append(dtm2.loc[i, 'Firm'])

In [71]:
empty

['삼우씨엠건축사사무소', '클루넷', '피엔에스커튼월', '후이즈글로벌네트워크']

In [93]:
dtm3 = dtm2[~dtm2['Firm'].isin(empty)]
dtm3 = dtm3.reset_index(drop = True)
dtm3

,Firm,가감,가게,가격결정,가격상한,가격연동제,가격차별,가격파괴,가격표,가결,...,힐,힐러리,힐링,힐스,힐스테이트,힐튼,힘,힘겨루기,힘줄,힙합
0,3S,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AJ네트웍스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AK홀딩스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,APS홀딩스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AP시스템,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2824,흥국화재,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2825,흥아해운,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2826,희림,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2827,힐세리온,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [95]:
# dtm3 저장
dtm3.to_pickle("2021_dtm_binary")

In [7]:
# dtm3 불러오기
dtm3 = pd.read_pickle("2021_dtm_binary")
dtm3

,Firm,가감,가게,가격결정,가격상한,가격연동제,가격차별,가격파괴,가격표,가결,...,힐,힐러리,힐링,힐스,힐스테이트,힐튼,힘,힘겨루기,힘줄,힙합
0,3S,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AJ네트웍스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AK홀딩스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,APS홀딩스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AP시스템,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2824,흥국화재,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2825,흥아해운,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2826,희림,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2827,힐세리온,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [8]:
# cosine similarity matrix 만들 때 계산 시간 단축하기 위한 작업
dic = {}
for i in range(len(dtm3)):
    dic[dtm3.loc[i, 'Firm']] = list(dtm3.loc[i, :].values)[1:]

In [ ]:
with open('dic.pkl', 'wb') as f:
    pickle.dump(dic, f)

# Cosine Similarity Matrix

In [27]:
from numpy import dot
from numpy.linalg import norm

In [31]:
# Cosine similarity matrix
firms = list(dtm3['Firm'])
csm = pd.DataFrame(index = firms, columns = firms)
csm

,3S,AJ네트웍스,AK홀딩스,APS홀딩스,AP시스템,AP위성,BGF,BGF리테일,BNGT,BNK캐피탈,...,휴코드홀딩스,흥구석유,흥국,흥국생명보험,흥국에프엔비,흥국화재,흥아해운,희림,힐세리온,힘스
3S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AJ네트웍스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK홀딩스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
APS홀딩스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AP시스템,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
흥국화재,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
흥아해운,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
희림,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
힐세리온,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
for i in firms:
    for j in firms:
        if i == j:
            csm.loc[i, j] = 1
        elif firms.index(i) < firms.index(j):
            csm.loc[i, j] = round(dot(dic[i], dic[j]) / (norm(dic[i]) * (norm(dic[j]))), 4)
        elif firms.index(i) > firms.index(j):
            csm.loc[i, j] = csm.loc[j, i]

In [ ]:
# csm 불러오기
csm = pd.read_excel('2021_csm.xlsx', index_col = 'Firm')
csm